In [1]:
import sqlite3
import time
import pandas as pd

from sqlite3 import Error
from pprint import pprint

# https://medium.com/alpha-vantage/get-started-with-alpha-vantage-data-619a70c7f33a
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators

# Import API key
from api_keys import api_key

In [2]:
def storeData(df_Stock):
    try:
        # store the ticker sykmbol
        ticker = df_Stock["ticker"][0]
        
        # get our connection
        conn = sqlite3.connect(r"static/data/stocks.sqlite")
        
        # create our cursor
        cur = conn.cursor()
        
        # first check to see if our table exists
        cur.execute("SELECT count(name) FROM sqlite_master WHERE type='table' AND name='stocks'")

        # if the count is 1, then our table exists
        if cur.fetchone()[0]==1:
            cur.execute(f"DELETE FROM stocks WHERE ticker = '{ticker}'")
        # end if
        
        # add this stock data to our database
        df_Stock.to_sql(name="stocks", con=conn, if_exists="append", index=True)
    
    except Error as e:
            print(f"Save failed for the stock: '{ticker}'. Please try again later./n/n  Error is:  {e}")
    finally:
        # make sure we close our connection
        conn.close()
    # end try
# end storeData()

In [8]:
def refresh():
    old_to_new = {"1. open": "open", "2. high": "high", "3. low": "low", "4. close": "close", "5. volume": "volume"}
    reorder = ["date", "ticker", "open", "high", "low", "close", "volume"]
    # Create a list of ticker symbols
    stocks = ["DAL", "AAL", "LUV", "UAL", "MSFT", "AAPL", "V", "INTC", "UBER", "BKNG", "LYFT", "TCOM", "ZM"]
    
    for stockTicker in stocks:
        print(f"Getting data for ticker:'{stockTicker}'")
        
        try:
            # Chose your output format, or default to JSON (python dict)
            ts = TimeSeries(api_key, output_format='pandas')
            
            # aapl_data is a pandas dataframe, aapl_meta_data is a dict
            stockData, meta_data = ts.get_daily(symbol=stockTicker)
            
            # add the ticker column
            stockData["ticker"] = stockTicker
            
            # because "date" is not a field but rather an index
            # reset the index so "date" is a field
            stockData.reset_index(inplace=True)
            
            # rename the columns
            stockData.rename(columns=old_to_new, inplace=True)
            
            # reorder the columns
            stockData = stockData[reorder]  
            
            # convert to datetime
            stockData["date"] = pd.to_datetime(stockData["date"])
            
            # set the index
            stockData.set_index("date", inplace=True)
            
            # convert the numeric fields to float
            stockData[["open", "high", "low", "close", "volume"]] = stockData[["open", "high", "low", "close", "volume"]].apply(pd.to_numeric)
            
            # stick all the data into our SQLite database
            storeData(stockData)

            time.sleep(12)
        except Error as e:
            print(f"The API failed for the stock '{stockTicker}'. Please try again later./n/n  Error is:  {e}")
        finally:
            print("The update process has completed.")
        # End Try
    # Next stockTicker
# End refresh()

In [10]:
refresh()

Getting data for ticker:'DAL'
The update process has completed.
Getting data for ticker:'AAL'
The update process has completed.
Getting data for ticker:'LUV'
The update process has completed.
Getting data for ticker:'UAL'
The update process has completed.
Getting data for ticker:'MSFT'
The update process has completed.
Getting data for ticker:'AAPL'
The update process has completed.
Getting data for ticker:'V'
The update process has completed.
Getting data for ticker:'INTC'
The update process has completed.
Getting data for ticker:'UBER'
The update process has completed.
Getting data for ticker:'BKNG'
The update process has completed.
Getting data for ticker:'LYFT'
The update process has completed.
Getting data for ticker:'TCOM'
The update process has completed.
Getting data for ticker:'ZM'
The update process has completed.


In [11]:
# TEST - pull data from our database to make sure it's all there and type casted correctly
# get our connection
conn = sqlite3.connect(r"static/data/stocks.sqlite")

cur = conn.cursor()
cur.execute("select * from stocks")
myDF = pd.DataFrame(cur.fetchall(), columns=["date", "ticker", "open", "high", "low", "close", "volume"])
conn.close()

print(myDF)

                     date ticker     open   high    low  close      volume
0     2020-04-16 00:00:00    DAL  23.7600  23.80  22.63  22.78  55973221.0
1     2020-04-15 00:00:00    DAL  24.9100  25.50  23.31  24.35  87798348.0
2     2020-04-14 00:00:00    DAL  23.9899  25.29  23.83  24.54  58599118.0
3     2020-04-13 00:00:00    DAL  24.9800  25.06  22.08  23.25  75904867.0
4     2020-04-09 00:00:00    DAL  25.4000  25.53  23.35  24.39  90591165.0
...                   ...    ...      ...    ...    ...    ...         ...
1295  2019-11-27 00:00:00     ZM  75.8100  75.84  73.36  74.08   1356274.0
1296  2019-11-26 00:00:00     ZM  75.0000  75.77  74.76  75.16   1848973.0
1297  2019-11-25 00:00:00     ZM  73.8000  75.06  72.86  74.55   1943187.0
1298  2019-11-22 00:00:00     ZM  72.0000  74.00  71.15  73.16   3059878.0
1299  2019-11-21 00:00:00     ZM  71.5000  71.95  69.58  69.92   1129235.0

[1300 rows x 7 columns]


In [ ]:
print(stockData)